In [287]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
# load data
df = pd.read_csv('rawdata_2007.csv')

In [289]:
# filtering dataset to leave target sample - adults who have private insurance
df.head()
df = df[(df['HIPRIVATEE'].isin([2,3])) & (df['ASTATFLG']==1)].copy()
df.drop(columns=['HIPRIVATEE','ASTATFLG'],inplace=True)

In [290]:
variables = df.columns

med_care = ['HOSPNGHT','SAWMENT','DELAYCOST']
med_condition = ['ARTHGLUPEV','ASTHMAEV','CANCEREV','CHEARTDIEV','CHOLHIGHEV','DEMENTIAEV','DIABETICEV','HEARTATTEV','HYPERTENEV','STROKEV','HRAUSES']
binary_vars = ['HIMCAIDE','HIMCAREE','CITIZEN','GOTWELF'] + med_care + med_condition

continuous_vars = ['AGE','BMICALC','HOURSWRK', 'HIP1COST', 'HIP2COST']
categorical_vars = ['REGION','SEX', 'MARSTCUR', 'FAMKIDNO', 'RACENEW', 'EDUC', 'EMPSTAT', 'POVERTY', 'HEALTH', 'DVINT', 'ERYRNO', 'ALCANYTP', 'SMOKFREQNOW', 'LAMTWRK']

In [291]:
mappings = {
    'REGION': {
        1: 'Northeast',
        2: 'North Central/Midwest',
        3: 'South',
        4: 'West'
    },
    'SEX': {
        1: 'Male',
        2: 'Female'
    },
    'MARSTCUR': {
        1: 'Married, spouse present',
        2: 'Married, spouse absent',
        3: 'Married, spouse in household unknown',
        4: 'Separated',
        5: 'Divorced',
        6: 'Widowed',
        7: 'Living with partner',
        8: 'Never married'
    },
    'FAMKIDNO':{
        0: 0,
        1: 1,
        2: 2,
        **{i : 3 for i in list(range(3,10))}
    },
    'RACENEW': {
        100: 'White only',
        200: 'Black/African American only',
        300: 'American Indian/Alaska Native only',
        400: 'Asian only',
        **{i: 'Other' for i in [510,530,541,542] + list(np.arange(997,1000))}
    },
    'EDUC': {
        103:'Grades 1-11',
        **{i:'Grades 1-11' for i in np.arange(104,116)},
        116:'12th grade, no diploma',
        201: 'High school graduate',
        202: 'GED or equivalent',
        301: 'Some college, no degree',
        302: 'AA degree: technical/vocational/occupational',
        303: 'AA degree: academic program',
        400: "Bachelor's degree (BA,AB,BS,BBA)",
        501: "Master's degree (MA,MS,Med,MBA)",
        **{i:'Professional School or Doctoral Degree (MD, DDS, DVM, JD, PhD, EdD)' for i in [502,503,505]}
    },
    'EMPSTAT': {
        **{i: 'Employed' for i in [100,110,111]},
        **{i: 'Not Employed' for i in [112,200,220] + list(range(120,123))+ list(range(200,218))}
    },
    'POVERTY': {
        10: 'Less than 1.0',
        11: 'Under 0.50',
        12: '0.50 to 0.74',
        13: '0.75 to 0.99',
        # 14: 'Less than 1.0 (no other detail)',
        20: '1.00 to 1.99',
        21: '1.00 to 1.24',
        22: '1.25 to 1.49',
        23: '1.50 to 1.74',
        24: '1.75 to 1.99',
        # 25: '1.00 - 1.99 (no further detail)',
        30: '2.00 and over',
        31: '2.00 to 2.49',
        32: '2.50 to 2.99',
        33: '3.00 to 3.49',
        34: '3.50 to 3.99',
        35: '4.00 to 4.49',
        36: '4.50 to 4.99',
        37: '5.00 and over',
        # 38: '2.00 and over (no other detail)'
    },
    'HEALTH': {
        1: 'Excellent',
        2: 'Very Good',
        3: 'Good',
        4: 'Fair',
        5: 'Poor'
    },
    'DVINT': {
        100: 'Never',
        200: 'Less than 1 year',
        201: 'Visits in the past 2 weeks',
        202: '2 weeks to less than 6 months',
        203: 'Under 6 months',
        204: '6 months to less than 12 months',
        300: '1 year or more',
        301: '1 year',
        302: '1 year to less than 2 years',
        303: '1 year to less than 3 years',
        304: '2 years to less than 3 years',
        305: '2 years to less than 5 years',
        306: '2 years',
        307: '2 to 4 years',
        308: '3 to 4 years',
        309: '3 years to less than 5 years',
        310: 'More than 3 years',
        400: '5 years or more',
        401: '5 to 9 years',
        402: 'More than 10 years'
    },
    'ERYRNO': {
        10: 'No visits',
        20: '1 visit',
        30: '2 to 3 visits',
        31: '2 visits',
        32: '3 visits',
        40: '4 or more visits',
        41: '4 to 9 visits',
        42: '4 to 5 visits',
        43: '6 to 7 visits',
        44: '8 to 9 visits',
        45: '10 to 12 visits',
        46: '13 or more visits',
        47: '13 to 15 visits',
        48: '16 or more visits'
    },
    'ALCANYTP': {
        **{i: 'Never/None' for i in [0,5]},
        1: 'Every day',
        2: 'Week',
        3: 'Month',
        4: 'Year'
    },    
    'SMOKFREQNOW': {
        **{i: 'Not at all' for i in [0,1]},
        2: 'Some days',
        3: 'Every day',
    },
    'LAMTWRK':{
        1: 'Not limited in work',
        2: 'Limited in kind/amt of work',
        3: 'Unable to work'
    },
    'HIMILITE':{
        10: 0,
        **{i:1 for i in [25,26]}
    },
    'GOTWELF': {
        10: 0,
        **{i: 1 for i in [20,21,22]}
    },   
    **{i: {
        1: 0,
        2: 1,
        3: 1
    } for i in ['HIMCAIDE','HIMCAREE']},
    **{i: {
        1: 0,
        2: 1,
        3: 0 # this only applies for DIABETICEV since 3 is coded as 'borderline' diabetes (fine to include here because no other binary vars have value of 3)
    } for i in med_care + med_condition + ['CITIZEN']} 
}

df['AGE'] = df['AGE'].replace([997,999],np.nan)
df['HOURSWRK'] = df['HOURSWRK'].replace([97, 98, 99],np.nan)
df['HIP1COST'] = df['HIP1COST'].replace([99997, 99998, 99999],np.nan)
df['HIP2COST'] = df['HIP2COST'].replace([99997, 99998, 99999],np.nan)
df = df[df['BMICALC'] != 996]

In [292]:
for i in binary_vars + categorical_vars:
    df[i] = df[i].map(mappings[i])

In [293]:
num_samples = df.shape[0]

# replacing "Unknown-refused", "Unknown-not ascertained", and "Unknown-don't know" responses with N/A
df.loc[df['HIP1COST'].isin([99997,99998,99999]),'HIP1COST'] = np.nan
df.loc[df['HIP2COST'].isin([99997,99998,99999]),'HIP2COST'] = np.nan

# COUNTING # OF 'UNKNOWN' RESPONSES FOR HIP1COST AND/OR HIP2COST
num_na_hip1cost = df['HIP1COST'].isna().sum()
num_na_hip2cost = df['HIP2COST'].isna().sum()
print(f"{num_na_hip1cost}({(100*num_na_hip1cost/num_samples):.2f}%) individuals responded 'Unknown' when asked for the out-of-pocket premium cost of their primary insurance plan.")
print(f"{num_na_hip2cost}({(100*num_na_hip2cost/num_samples):.2f}%) individuals responded 'Unknown' when asked for the out-of-pocket premium cost of their secondary insurance plan.")

# total number of rows for which HIP1COST or HIP2COST is unknown
num_na_hipcost = (df['HIP1COST'].isna() | df['HIP2COST'].isna()).sum()
print(f"{num_na_hipcost}({(100*num_na_hipcost/num_samples):.2f}%) individuals responded 'Unknown' for either out-of-pocket premium cost of their primary or secondary insurance plan.")

2020(15.35%) individuals responded 'Unknown' when asked for the out-of-pocket premium cost of their primary insurance plan.
92(0.70%) individuals responded 'Unknown' when asked for the out-of-pocket premium cost of their secondary insurance plan.
2077(15.79%) individuals responded 'Unknown' for either out-of-pocket premium cost of their primary or secondary insurance plan.


In [294]:
# MISSINGNESS - dropping the rows for which HIP1COST or HIP2COST is unknown 
df.drop(df[(df['HIP1COST'].isna() | df['HIP2COST'].isna())].index,inplace=True)

In [295]:
# combining HIP1COST and HIP2COST into one response variable for the total out-of-pocket cost
df['HIPCOST'] = df['HIP1COST']+df['HIP2COST']

In [296]:
df.reset_index(inplace=True,drop=True)

In [297]:
df.to_csv('cleaned_data2007.csv')